# 課題3: 分類モデルの作成

本課題ではseabornに添付の「titanic」データセットを読み込み、生存者を予測します。各セルに入っているコメントの下に、実行するコードを記入してください。わからない場合は、ここまでのレッスン内容や各種ライブラリの公式ドキュメントを参照しましょう。

## 1. 必要なモジュールのインポート

In [93]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.model_selection import train_test_split

# JupyterNotebook上でグラフを表示する設定
%matplotlib inline
# DataFrameで全ての列を表示する設定
pd.options.display.max_columns = None

## 2. データの読み込み

seabornに添付のデータセットから「titanic」を読み込み、内容を確認します。

In [94]:
dataset = sns.load_dataset("titanic")

### 使用する列の指定
今回は「survived, pclass, sex, age, sibsp, parch, fare, embarked」の列を使用します。
#### 参考:全ての列の説明
- survived: 生存区分(0:死亡, 1:生存)
- pclass: チケットクラス
- sex: 性別(male:男性, female:女性)
- age: 年齢
- sibsp: 同乗している兄弟や配偶者の数
- parch: 同乗している親や子供の数
- fare: 料金
- embarked: 乗船した港(頭文字)
- class: 客室クラス
- who: 性別(man:男性, woman:女性)
- adult_male: 成人男性ならTrue
- deck: 事故の際にどのデッキにいたか
- embark_town: 乗船した港名
- alive: 生存区分(no:死亡, yes:生存)
- alone: 1人で乗船したか

In [95]:
# datasetから「survived, pclass, sex, age, sibsp, parch, fare, embarked」の列を取得してdatasetに代入する
dataset = dataset[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]

In [96]:
# datasetのデータの最初の5行を表示
dataset.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


## 3. データの前処理

### 要約統計量の表示

In [97]:
# 要約統計量を表示
dataset.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### 欠損値の確認と補完
#### 欠損値の確認

In [98]:
# 各列の欠損値の数を確認
dataset.isnull().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
dtype: int64

#### 欠損値の補完
ageの欠損値を平均値にて補完

In [99]:
# ageの欠損値を平均値にて補完
dataset['age'] = dataset['age'].fillna(dataset['age'].mean())

embarkedの欠損値をもっとも乗船者数の多い港にて補完

- Series.value_counts:分類ごとの件数を取得する

<a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html" target="_blank">参考:value_counts</a>

In [100]:
# 乗船者数の多い港を value_counts メソッドで確認
dataset['embarked'].value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [101]:
# embarkedの欠損値をもっとも乗船者数の多い港にて補完
dataset['embarked'] = dataset['embarked'].fillna(dataset['embarked'].mode().iloc[0])

#### 補完後の欠損値の確認

In [102]:
# 補完後の欠損値が0であることを確認
dataset.isnull().sum()

survived    0
pclass      0
sex         0
age         0
sibsp       0
parch       0
fare        0
embarked    0
dtype: int64

### ダミー変数への変換
sexとembarkedをダミー変数に変換

In [103]:
# datasetのsexとembarkedをダミー変数に変換してdataset2に代入する
dataset2 = pd.get_dummies(dataset, columns=['sex', 'embarked'])

In [104]:
# dataset2のデータの最初の5行を表示
dataset2.head()

,survived,pclass,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1


## 4. 目的変数と説明変数の選択

ここでは、以下の列を使用します。
- 目的変数: `survived`
- 説明変数: それ以外

列の除外には drop を使います。
```
データフレーム.drop(columns=除外したい列名)
```

In [105]:
# dataset2より目的変数と説明変数に該当する列を取得してnumpy配列に変換し、変数YとXに格納する
# Y:目的変数に該当する列
Y = np.array(dataset2['survived'])
# X:説明変数に該当する列。dataset2からsurvivedを除外
X = np.array(dataset2.drop(columns=['survived']))

### YとXの形状を確認

In [106]:
# 形状を確認
print(Y.shape)
print(X.shape)

(891,)
(891, 10)


## 5. データの分割

この課題ではホールドアウト法でデータを分割します。

In [107]:
# X と Y を 機械学習用データとテストデータに7:3に分ける(X_train, X_test, Y_train, Y_test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [108]:
# 機械学習用データを、学習データと検証データに7:3に分ける(X_train, X_valid, Y_train, Y_valid)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.3, random_state=0)

In [109]:
# 形状を確認:X_train, X_valid, X_test, Y_train, Y_valid, Y_test
print("X_train=", X_train.shape, ", Y_train=", Y_train.shape)
print("X_valid=", X_valid.shape, ", Y_valid=", Y_valid.shape)
print("X_test=", X_test.shape, ", Y_test=", Y_test.shape)

X_train= (436, 10) , Y_train= (436,)
X_valid= (187, 10) , Y_valid= (187,)
X_test= (268, 10) , Y_test= (268,)


## 6. モデルの選択

ロジスティック回帰と決定木、ランダムフォレスト、SVMの4つのモデルを作成して比較します。

In [110]:
# 必要なライブラリのインポート
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score

In [111]:
# 空のリストを用意
model_list = []

# リストにモデルを追加。それぞれの引数は警告を出さないための設定
model_list.append(LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000))
model_list.append(DecisionTreeClassifier(criterion='entropy'))
model_list.append(RandomForestClassifier(n_estimators=100))
model_list.append(SVC(gamma='scale'))

# for文でリストからモデルを取り出し、学習と予測、F1値の出力を行う
for model in model_list:
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    print(f1_score(Y_test, Y_pred))

0.7425742574257426
0.6666666666666666
0.717948717948718
0.47297297297297297


## 7. パラメータのチューニング

GridSearchCVを使い、性能の良かったランダムフォレストのパラメータのチューニングを行います。

In [112]:
# 必要なライブラリのインポート
from sklearn.model_selection import GridSearchCV

In [113]:
# モデル:ランダムフォレスト
model = RandomForestClassifier()

# パラメータ:n_estimators:木の数、max_depth:木の深さ
params = {'n_estimators':[10, 50, 100], 'max_depth':[5, 10, 50]}

# グリッドサーチ
gscv = GridSearchCV(model, param_grid=params, cv=3)

# データの分割:機械学習用データを学習と検証に分けるのはクロスバリデーションで行ってくれる
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

# fit
gscv.fit(X_train, Y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 10, 50],
                         'n_estimators': [10, 50, 100]})

In [114]:
# 最適なパラメータを表示
gscv.best_score_

0.8186392914653785

### 最適なパラメータをもとにモデルを構築
#### データの分割

In [115]:
# データの分割
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.3, random_state=0)

#### 最適なパラメータによる学習と評価

In [116]:
# 最適なパラメータによる学習
randomforest_model = RandomForestClassifier(n_estimators=100, max_depth=5)
randomforest_model.fit(X_train, Y_train)
Y_pred = randomforest_model.predict(X_valid)

In [117]:
# F1値の出力
print(f1_score(Y_valid, Y_pred))

0.7343750000000001


## 8. テストデータによる汎化性能の確認

最後にテストデータでモデルの汎化性能を確認しましょう

In [118]:
# テストデータを使って予測を行いF1値を算出
Y_pred = randomforest_model.predict(X_test)
print(f1_score(Y_test, Y_pred))

0.7340425531914891
